<a href="https://colab.research.google.com/github/screid/Estadistica_Computacional_UGM/blob/main/Codigo_para_obtener_georeferencias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopy

In [ ]:
# Librerias
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# 1. Funciones (no tocar)

In [10]:
def escritura_y_transformacion_dataframe(data, comuna=None, region=None, n_partes=1, sobrescribir_ubicacion=True):
    """
    Si sobrescribir_ubicacion=True, modifica o agrega las columnas COMUNA, REGION y PAIS.
    Si es False, deja las columnas como están.
    """
    if sobrescribir_ubicacion:
        if comuna is None or region is None:
            raise ValueError("Debes proporcionar 'comuna' y 'region' si deseas sobrescribir ubicación.")
        data['COMUNA'] = comuna
        data['REGION'] = region
        data['PAIS'] = 'Chile'

    lista_de_dataframes = np.array_split(data, n_partes)

    dataframe_dict = {}
    for i, df_parte in enumerate(lista_de_dataframes):
        dataframe_dict[f'df_parte_{i+1}'] = df_parte

    return dataframe_dict

In [11]:
def funcion_calculadora_de_lat_lon(data):

  # Función para limpiar la dirección
  def limpiar_direccion(direccion):
    if pd.isnull(direccion):
        return None
    direccion = str(direccion).upper()
    direccion = re.sub(r'\b(LOCAL|local|OFICINA|oficina|OF.|of.|DEPTO|depto|DPTO|dpto|DEPARTAMENTO|departamento|S/N|s/n|SITIO|sitio|ST.|st.|SECTOR|sector|CMNO|cmno|CASA|casa|PISO|piso|CARAMPANGUE|ARAUCO)\b[\s\S]*', '', direccion)
    direccion = direccion.strip()
    return direccion if direccion else None


  # Construir dirección completa para geocodificar
  def construir_direccion_completa(row):
    base = limpiar_direccion(row['DIRECCION'])
    if base is None:
        return None  # dirección inválida, no se geocodifica
    comuna = row['COMUNA']
    region = row['REGION']
    pais = row['PAIS']
    return f"{base}, {comuna}, {region}, {pais}"

  # Aplicar transformación
  data['DIRECCION_GEO'] = data.apply(construir_direccion_completa, axis=1)

  # Configurar geocoder
  geolocator = Nominatim(user_agent="macaco", timeout=5)
  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=1, error_wait_seconds=1.0)

  # Función de geocodificación robusta
  def obtener_coordenadas(direccion):
      try:
          ubicacion = geocode(direccion)
          if ubicacion:
              return pd.Series([ubicacion.latitude, ubicacion.longitude])
          else:
              return pd.Series([None, None])
      except Exception as e:
          print(f"Error con dirección '{direccion}': {e}")
          return pd.Series([None, None])

  # Aplicar
  data[['latitud', 'longitud']] = data['DIRECCION_GEO'].apply(obtener_coordenadas)

  return data

# 2. Ejecución

## 2.1. Cargar dataframe y visualización del mismo

In [12]:
# Cargas el dataframe
# MODIFICAS AQUI Y  PONES EL NOMBRE DEL ARCHIVO
data = pd.read_csv('Patentes Antuco.csv', sep = ';')

In [13]:
# para mostrar las primeras 5 filas
data.head(5)

,Rol,Tipo de Patente,Nro. Giro,Rut,Nombres,DIRECCION,Rubro
0,1-00001,Industrial,102228,5.763.975-K,BURGOS CASTILLO MARTA RAQUEL,LOS CARRERAS 433,FABRICA DE PAN
1,1-00004,Industrial,102285,91.081.000-6,ENEL GENERACION CHILE S.A,CENTRALES DEL LAJA,COMPAÑIA GENERADORA DE ELECTRICIDAD
2,1-00005,Industrial,102212,79.883.210-7,FORESTAL COMACO S.A.,FUNDO LAS MELLIZAS,EXPLOTACION DE MADERA
3,1-00006,Industrial,102287,96.954.810-0,SUR INVERSIONES S.A,NaN,REPRODUCCION DE PECES Y MARISCOS
4,1-00007,Industrial,102298,9.379.613-6,ASTETE TARIFENO YANET DE LAS MERCEDES,380,PRODUCTOS ARTESANALES


In [14]:
print("Número de filas: " , data.shape[0])

Número de filas:  167


## 2.2. Ejecución del código.
Recuerda que si necesitas correr por dataframe particionado, ocupas la primera fila. Si no, ocupas la segunda.

In [ ]:
# UTILIZA ESTE CODIGO SI NECESITAS DIVIDIR EL DATAFRAME
# Solo modificar aquí si se desea cambiar el número de partes
n_partes = 6

# Ejecuta todo
partes = escritura_y_transformacion_dataframe(
    data,
    comuna='Providencia',
    region='Metropolitana',
    n_partes=n_partes,
    sobrescribir_ubicacion=True
)

for nombre, df_parte in partes.items():
    print(f"Procesando {nombre}...")
    resultado = funcion_calculadora_de_lat_lon(df_parte)
    resultado.to_csv(f"{nombre}_resultado.csv", index=False)

In [ ]:
# UTILIZA ESTE CODIGO SI NO NECESITAS DIVIDIR EL DATAFRAME
# Procesamiento sin dividir en partes
resultado = escritura_y_transformacion_dataframe(
    data,
    comuna='Antuco',
    region='Biobio',
    n_partes=1,
    sobrescribir_ubicacion=True
)['df_parte_1']

# Aplicar cálculo de coordenadas
resultado = funcion_calculadora_de_lat_lon(resultado)

# Guardar resultado
resultado.to_csv("resultado_Patentes Antuco.csv", index=False)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
